# Deep Learning: More Convolutional Neural Networks

Welcome back! Today’s small–group lecture explores **Convolutional Neural Networks (CNNs)**, building directly on what we've learned so far in deep learning.

CNNs are the workhorse behind many modern computer vision systems — from image classification (e.g., recognizing cats vs. dogs) to object detection (e.g., detecting pedestrians in self-driving cars) and even style transfer.

In a standard fully-connected neural network, every neuron in one layer connects to every neuron in the next. That works well for tabular data, but it ignores the fact that **images have spatial structure**: nearby pixels tend to be related. CNNs exploit this by using small filters (kernels) that slide over the image to detect local patterns like edges and textures.

By the end of this session, you will be able to:
- Understand the intuition behind CNNs and why they work so well for image data.
- Build a multi-layer PyTorch CNN from scratch.
- Explain the role of activation functions, batch normalization, and max pooling.
- Train, validate, and analyze a CNN’s performance.
- Understand core training concepts such as optimizers, loss functions, epochs, and learning rate.

This lecture is designed to feel like a guided walkthrough — with explanations of not just **what** we do, but **why** we do it. As you go, try to connect the code to the mathematical ideas from lecture.

Let's begin!

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

print(f"Pytorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

Pytorch version: 2.3.1.post100
Torchvision version: 0.18.1


### Note: Python must be 3.11.x

If your kernel shows a value > 3.11.x you will need to downgrade. Please email the staff for help.

**Why does this matter?** Deep learning frameworks like PyTorch provide precompiled binaries (called *wheels*) for specific Python versions and hardware setups. When Python introduces a new major/minor version (like 3.12), the internal binary interface (ABI) can change. If PyTorch hasn't released a compatible wheel yet, you may see mysterious runtime errors, crashes, or import failures.

So as a rule of thumb:
- Always check the PyTorch install page for supported Python versions.
- If you see version mismatches, try creating a **conda or venv environment** with Python 3.11 specifically for deep learning work.

In this class, we standardize on **Python 3.11.x** so the environment is reproducible and supportable.

In [2]:
# DEVICE CONFIGURATION
if torch.backends.mps.is_available():          # Apple Silicon (Metal Performance Shaders)
    device = torch.device("mps")
elif torch.cuda.is_available():                # NVIDIA GPU with CUDA
    device = torch.device("cuda")
else:
    device = torch.device("cpu")               # Fallback to CPU

print("Using device:", device)

Using device: mps


For today's small group, we will walk through the process of setting up a convolutional neural network ("CNN" for short) using the `pytorch` package!

CNNs shine when working with **structured grid data**, especially images — tasks like classification, segmentation, object detection, and more.

Why? Because CNNs:
- Capture local patterns (edges, textures) via **convolutions**.
- Build hierarchical features (shapes → object parts → full objects) through **stacked layers**.
- Use **shared weights**, meaning the same filter is applied across the whole image, which makes them parameter-efficient and **translation-equivariant** (shifting the image shifts the feature map in a predictable way).

A quick mental picture:
- Early layers learn to detect **edges** and simple textures.
- Middle layers detect **parts** like eyes, wheels, or wings.
- Later layers detect **semantic concepts** like "dog", "car", or "bird".

Let’s load a dataset so we can see these ideas in action.

Recall from lecture that CNNs are generally used to process gridded data or images.

Let's begin by loading one of the toy datasets included in `pytorch`: **CIFAR-10**.

The dataset contains 60,000 small 32×32 color images belonging to 10 different classes:
- airplane
- automobile
- bird
- cat
- deer
- dog
- frog
- horse
- ship
- truck

There are 50,000 training images and 10,000 test images. Each image has **3 color channels (RGB)** and a relatively low resolution (32×32), which makes it great for teaching and small experiments.

We will also apply a **transform pipeline** to preprocess the images before feeding them into the network.

In [3]:
# init preprocessing for CIFAR-10 dataset (images are 32x32x3)
transform = transforms.Compose([
    transforms.ToTensor(),
    # ToTensor() converts images from [0, 255] uint8 to [0.0, 1.0] float32
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize each channel to roughly [-1, 1]
])

In [4]:
batch_size = 100
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Great! We have image data now. But what does it look like?

Visualizing your data is an essential first step — especially in computer vision. It's very easy to accidentally:
- Misinterpret labels.
- Apply the wrong preprocessing.
- Feed the network images that are rotated, upside-down, or poorly scaled.

Let's plot the different classes below using `matplotlib`. When teaching, this is a great moment to ask:
- *What patterns do you notice?* (e.g., background colors, viewpoints, clutter)
- *Which classes might be harder for the network? Why?* (e.g., cat vs. dog may be harder than airplane vs. frog)

We won't fill in the plotting code here, but you are encouraged to:
- Sample a batch from `train_loader`.
- Show a grid of images.
- Print the corresponding class names.

In [5]:
# Plot each class here

## Building a CNN

Staff:
- Reference lecture: CNNs involve stacking multiple **layers**
- The first part of a CNN involves stacking multiple layers of convolutional, activation, and maxpool layers 
- The example code below shows 3 of these stacks of layers!
<p align="left">
    <img src = "https://media.geeksforgeeks.org/wp-content/uploads/20250529121802516451/Convolutional-Neural-Network-in-Machine-Learning.webp" width = "500">
</p>

Conceptually, our CNN will:
1. Take a 3×32×32 image.
2. Apply a series of convolution + nonlinearity + pooling operations that gradually reduce spatial size but increase the number of feature maps.
3. Flatten the final feature maps into a vector.
4. Feed that vector into a fully connected layer that outputs **logits** for the 10 CIFAR-10 classes.

### Activation Functions

Staff:
- Please briefly review some of the common activation functions (there will be a table on this at the beginning of lecture)
- Discuss 3 of the most common activation functions
- Be sure to define what their names are in tensorflow

### Expanded Notes for Staff
- **ReLU** (Rectified Linear Unit): `f(x) = max(0, x)`
  - Very simple, fast to compute.
  - Avoids vanishing gradients on the positive side.
  - TensorFlow: `tf.nn.relu` or `tf.keras.layers.ReLU()`.

- **LeakyReLU**: `f(x) = x` if `x > 0`, and `αx` otherwise (α is small, like 0.01).
  - Solves the "dying ReLU" problem by allowing a small negative gradient.
  - TensorFlow: `tf.nn.leaky_relu`.

- **Tanh**: squashes values to (-1, 1).
  - Zero-centered, which can help optimization.
  - Still suffers from saturation/vanishing gradients at large |x|.
  - TensorFlow: `tf.nn.tanh` or `tf.keras.activations.tanh`.

Explain *why* we use nonlinear activations: without them, a stack of layers collapses to a single linear transformation — so no matter how many layers you add, the overall function is still just a linear mapping. Nonlinear activations give the network the ability to approximate **complex nonlinear functions**.

Please add comments to this code:

In [6]:
# Define a simple CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Input shape: (batch_size, 3, 32, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),        # -> (batch_size, 32, 32, 32)
            nn.BatchNorm2d(32),                                # normalize activations per channel
            nn.ReLU(),                                         # nonlinear activation
            nn.MaxPool2d(2)                                    # -> (batch_size, 32, 16, 16)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),       # -> (batch_size, 64, 16, 16)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)                                    # -> (batch_size, 64, 8, 8)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),      # -> (batch_size, 128, 8, 8)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)                                    # -> (batch_size, 128, 4, 4)
        )
        # Flattened feature vector size: 128 * 4 * 4 = 2048
        self.fc = nn.Linear(128*4*4, 10)  # CIFAR-10 has 10 classes

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # flatten (batch_size, 128*4*4)
        out = self.fc(out)               # output logits (batch_size, 10)
        return out

model = CNN().to(device)

### Q: What do you think will happen to your CNN as you change the activation function?
Feel free to try this by changing the activation layers in `CNN`!

### A:
Different activation functions drastically change how gradients behave and how expressive the model is.

- **ReLU**:
  - Pros: fast, tends to work well out-of-the-box, avoids vanishing gradients on the positive side.
  - Cons: some neurons can "die" (output 0 for all future inputs) if weights push them too far into the negative region.

- **Sigmoid**:
  - Squashes values to (0, 1).
  - Gradients get very small for large |x| (saturation), leading to **vanishing gradients** and slow learning.
  - Often a poor choice for deep CNNs.

- **Tanh**:
  - Similar to sigmoid but output is centered at 0 in (-1, 1), which can help optimization somewhat.
  - Still suffers from saturation and vanishing gradients.

- **LeakyReLU**:
  - Keeps a small negative slope for x < 0, so neurons never fully die.
  - Often slightly more robust than plain ReLU.

So, if you switch from ReLU to sigmoid or tanh, you might observe:
- Slower learning.
- Worse final accuracy.
- Training that appears to “stall” early.

If you switch from ReLU to LeakyReLU, you might get slightly smoother training and sometimes a small performance boost, especially in deeper networks.

----
## Training a CNN
Staff:
- Please discuss the selection process for optimizer and loss inputs
- Define what learning rate is
- Give an overview of 2-3 common loss functions and their behavior

### Expanded Lecture Notes

Training a neural network is an instance of **iterative optimization**. The key components are:

1. **Loss function (objective)**: tells us how bad our predictions are.
2. **Optimizer**: decides how to update parameters to reduce the loss.
3. **Learning rate**: controls how big each update step is.
4. **Epochs and batches**: define how many passes we make over the dataset and how we break data into mini-batches.

### Loss function
- For multi-class classification like CIFAR-10, we treat the model’s output as **logits** (unnormalized scores) for each class and compare them to the correct class labels.
- `nn.CrossEntropyLoss()` in PyTorch:
  - Applies a softmax internally to convert logits to probabilities.
  - Computes the negative log-likelihood of the correct class.
  - Encourages the network to assign high probability to the correct label.

Other loss functions (for context):
- `nn.MSELoss()` (mean squared error): often used in regression, not ideal for classification because it doesn’t match the probabilistic structure of the problem.
- `nn.BCEWithLogitsLoss()`: used for binary classification or multi-label classification (where each class is independently 0 or 1).

### Optimizer
- **SGD (Stochastic Gradient Descent)**:
  - Update rule: `θ ← θ − η * ∇_θ L` (plus optional momentum).
  - Simple and effective but can require careful tuning of the learning rate and momentum.

- **Adam (Adaptive Moment Estimation)**:
  - Keeps moving averages of both gradients and squared gradients.
  - Adapts the step size for each parameter separately.
  - Often converges faster with less manual tuning, making it a great default choice.

### Learning rate
- The learning rate `η` determines the **step size** in parameter space.
- Too large: the loss can oscillate or blow up, because we overshoot the minimum.
- Too small: the network learns extremely slowly and may appear stuck.
- In practice, people often use **learning rate schedules** (decay over time) or **adaptive optimizers** like Adam.

[This](https://www.geeksforgeeks.org/machine-learning/epoch-in-machine-learning/) reference will be useful to review the concept of epochs if needed.

In [7]:
#  Loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Q: What might happen if we changed our loss from __ to __ ?

### A:
Let’s consider a few concrete changes:

1. **CrossEntropyLoss → MSELoss** for CIFAR-10 classification:
   - The model now tries to match one-hot vectors using squared error.
   - The gradients can become very small in regions where they should be large (softmax-like structure is lost).
   - Training tends to be slower and can converge to worse optima.

2. **CrossEntropyLoss → BCEWithLogitsLoss** without changing labels:
   - BCEWithLogitsLoss expects a different label format (typically floats 0/1 per class).
   - You would be pretending the 10 classes are independent binary labels instead of mutually exclusive.
   - This mismatch usually leads to incorrect behavior or outright shape errors.

3. **CrossEntropyLoss → a custom margin-based loss**:
   - Might encourage larger separation between the correct class logit and the others.
   - Could improve robustness but would need careful design.

In summary: **CrossEntropyLoss is the standard choice** for multi-class classification with mutually exclusive labels. Changing away from it without adjusting the problem formulation can significantly hurt performance.

### Q: What happens if the `learning_rate` parameter is too high? Or too low?

### A:
- **Too high learning rate**:
  - Loss may explode to very large values.
  - The training curve may bounce around and never settle.
  - We might see `nan` losses if the gradients become too large.

- **Too low learning rate**:
  - Training loss decreases very slowly.
  - It may seem like the model "is not learning", but really it’s just taking tiny steps.
  - You might stop early because it appears stuck near random-guess accuracy.

In practice, you often try a few learning rates on a small number of epochs and see how the loss curve behaves. This is sometimes called a **learning rate sweep** or **learning rate range test**.

In [11]:
# Training loop
# STAFF: Please add check for early stopping!!!
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()  # set model to training mode (enables dropout, batchnorm updates)
    running_loss = 0.0
    for i,(images, labels) in enumerate(tqdm(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()   # clear old gradients
        loss.backward()         # compute new gradients via backprop
        optimizer.step()        # update model parameters
        
        running_loss += loss.item()

    # Note: here we print the loss from the last batch of the epoch
    print(f'Loss: {loss.item():.4f} \n')

### Q: What happens if you increase `epochs`? Will performance always improve as `epochs` increases?

### A:
Performance improves early on, but after a certain number of epochs the model begins to **overfit**:
- Training accuracy continues to increase.
- Validation/test accuracy plateaus or starts to **decrease**.

Intuitively, the model begins to memorize specific details and noise from the training set instead of learning general patterns that transfer to new data.

Common strategies to combat overfitting include:
- **Early stopping**: monitor validation loss/accuracy and stop when it stops improving.
- **Regularization**: weight decay, dropout, data augmentation.
- **More data**: often the best solution if available.

----
## Validating a CNN

Staff: Please add comments/explanations as needed to this code!

We now want to evaluate how well our trained model **generalizes** to unseen data. This is why we kept a separate **test set**.

Key steps during evaluation:
- Call `model.eval()` to set the model to **evaluation mode**.
  - This tells layers like BatchNorm and Dropout to behave differently (e.g., BatchNorm uses running statistics instead of batch statistics, Dropout is disabled).
- Use `torch.no_grad()` to avoid tracking gradient computations.
  - This saves memory and speed, since we aren’t going to backpropagate.
- Loop over the `test_loader`, compute predictions, and count how many are correct.

The result is the **test accuracy**, which we treat as an estimate of how well our model will perform on new, real-world data from the same distribution.

In [12]:
# Testing
model.eval()  # set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # torch.max returns (max_value, index_of_max)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


----

## Analyzing Performance
- Staff: prompt some reflection about the plot below

A single test accuracy number is useful, but it doesn’t tell the whole story. Often, we want to understand **how performance changes over time** as we train longer.

Things to consider when looking at an accuracy-vs-epochs plot:
- Does training accuracy keep going up while test accuracy plateaus? → Likely overfitting.
- Is both training and test accuracy low and not improving? → Underfitting (model too simple, learning rate too low, not enough epochs, etc.).
- Do we see noisy, jagged curves? → Maybe a high learning rate or small batch size.

You can also:
- Plot both **training loss** and **test loss** vs. epoch.
- Check if there is a gap between training and test curves.
- Use such plots to decide when to stop training or when to adjust hyperparameters.

This is a great opportunity to discuss *hyperparameter tuning*, *model capacity*, and *training dynamics*.

In [ ]:
# Plot accuracy vs. epochs